In [1]:
import sys;sys.path.append('../')
import os

from rich import print

openai_key=open('../openai.key').read().strip()
os.environ['OPENAI_API_KEY'] = openai_key

from multi_agent_llm import OpenAILLM
from multi_agent_llm.agents.clu.split_clu import CLU

llm = OpenAILLM(model_name="gpt-4o-mini")

In [2]:
clu = CLU(main_role="""
    Your task is to learn how to reason step by step. Break down complex tasks into smaller, manageable sub-steps. 
    Extract intermediate conclusions at each step before arriving at the final answer. Dont learn anything task specific but learn in general 
    how to enhance your answer by thinking step by step and planning on how to go towards final answer.
    """,
    collection_name="step_by_step_knowledge_base_v1",
    compress_knowledge=False,
    retrival_limit=15,
    llm=llm,
    pruning_queue_size=1,
    exploration_rate=0.2,
    verbose=True,
)

In [3]:
from typing import List

from pydantic import BaseModel, Field


class Step(BaseModel):
    explanation: str = Field(..., description="Explanation for this step of reasoning")
    intermediate_answer: str = Field(..., description="Intermediate answer or conclusion for this step")

class QA(BaseModel):
    steps: List[Step] = Field(..., description="List of steps in the reasoning process")
    answer: str = Field(..., description="Final answer (A/B/C/D/Uncertain) without any additional explanation.")

In [ ]:
training_example = """
    Question: A man walks 5 miles north, then 3 miles east. How far is he from his starting point?
"""

# Train CLU with the step-by-step example
for i in range(1):
    result = clu.train(training_example, expected_output=QA(steps=[
        Step(explanation="Recognize the movement in two directions (north and east)", intermediate_answer="5 miles north and 3 miles east"),
        Step(explanation="Form a right triangle", intermediate_answer="Right triangle with sides 5 and 3"),
        Step(explanation="Apply Pythagorean theorem", intermediate_answer="sqrt(5^2 + 3^2) = sqrt(34)"),
        Step(explanation="Calculate the distance", intermediate_answer="5.83 miles")
    ], answer="5.83 miles"), response_schema=QA)
    print(result)

In [ ]:
# Infer the reasoning process learned by CLU
result = clu.inference("Explain how you reasoned through this problem step by step.")
print(result['response'])